## Entorno

In [25]:
!pip --version

pip 24.0 from C:\Users\aoran\Desktop\Clases\Octavo\Proyecto algo\Cloud\Act 2\venv\Lib\site-packages\pip (python 3.11)



In [26]:
!pip install azureml-sdk


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
!pip install pandas
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


^C



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Despliegue en la nube

In [ ]:
from azureml.core import Workspace

ws = Workspace.create(name="Actividad_2",
                      subscription_id = "id",
                      resource_group = "__hw2__",
                      location = "centralindia") #centralindia

Deploying KeyVault with name actividakeyvault3489cd54.
Deploying AppInsights with name actividainsightsc401386a.
Deployed AppInsights with name actividainsightsc401386a. Took 8.57 seconds.
Deploying StorageAccount with name actividastorage8d0086b75.
Deployed KeyVault with name actividakeyvault3489cd54. Took 25.31 seconds.
Deploying Workspace with name Actividad_2.
Deployed StorageAccount with name actividastorage8d0086b75. Took 30.45 seconds.
Deployed Workspace with name Actividad_2. Took 40.78 seconds.


In [40]:
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="modelo",
                                  workspace=ws)

Registering model modelo


In [41]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pandas',
    'scikit-learn'
    ])

In [42]:
import json

umb = open("umbral.json", "r")
umb = json.load(umb)
umbral = umb["umbral"][0]

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('modelo')
  model = joblib.load(model_path)

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)


    result = model.predict(data_dummies).tolist()
    result_sigmoid = sigmoid(result)
    umbral = {umbral}
    result_finals = [1 if x > umbral else 0 for x in result_sigmoid]

    return json.dumps(result_finals)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [48]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=6)

service = Model.deploy(workspace=ws,
                       name='bankrupcy',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\aoran\AppData\Local\Temp\ipykernel_9232\950746086.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [49]:
service.wait_for_deployment()

Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 04ce4b94-f988-4a62-8914-b69813257f79
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config.",
  "details": [
    {
      "code": "ScoreInitRestart",
      "message": "Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 04ce4b94-f988-4a62-8914-b69813257f79
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config.",
  "details": [
    {
      "code": "ScoreInitRestart",
      "message": "Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: 04ce4b94-f988-4a62-8914-b69813257f79\nMore information can be found using

In [ ]:
scoring_uri = service.scoring_uri

In [ ]:
scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()